# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [758]:
#📥 Import et configuration
import requests as r
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import json

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [759]:
### 💡 Première requête guidée
load_dotenv() # charge les clés de .env dans les variables d'environnement
API_KEY = os.getenv("OPENWEATHER_API_KEY")
BASE_URL = "https://api.openweathermap.org/data/2.5"

def get_weather_data(city, api_key):
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    # URL : current weather data
    url = f"{BASE_URL}/weather"

    # Paramètres à compléter
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric',  # Celsius
        'lang': 'fr'
    }

    try:
        response = r.get(url, params=params)
        response.raise_for_status() # Vérifie si status code est 200, sinon lève une exception

        data = response.json()
        return data
    except r.HTTPError as http_err:
        print(f"Http Error: {http_err}\nStatus code: {r.status_codes}")
    except r.Exception.ConnectionError as cn_error:
        print(f"Error: {cn_error}")
    return None 
    # Votre code ici pour l'appel API
    # Gérez les erreurs HTTP !
get_weather_data("Montpellier",API_KEY)
 # Remplacez par votre implémentation

{'coord': {'lon': 3.8772, 'lat': 43.6109},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'ciel dégagé',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 28.05,
  'feels_like': 27.85,
  'temp_min': 28.05,
  'temp_max': 28.71,
  'pressure': 1013,
  'humidity': 42,
  'sea_level': 1013,
  'grnd_level': 1004},
 'visibility': 10000,
 'wind': {'speed': 9.26, 'deg': 330},
 'clouds': {'all': 0},
 'dt': 1753454420,
 'sys': {'type': 2,
  'id': 2038454,
  'country': 'FR',
  'sunrise': 1753417576,
  'sunset': 1753470927},
 'timezone': 7200,
 'id': 2992166,
 'name': 'Montpellier',
 'cod': 200}

In [760]:
# Configuration API
API_KEY = os.getenv("OPENWEATHER_API_KEY")  # Remplacez par votre vraie clé
BASE_URL = "http://api.openweathermap.org/data/2.5"

# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Rome", "Amsterdam", "Vienna"]

get_weather_data(CITIES[4], API_KEY)

{'coord': {'lon': 4.8897, 'lat': 52.374},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'partiellement nuageux',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 22.74,
  'feels_like': 22.79,
  'temp_min': 21.6,
  'temp_max': 23.83,
  'pressure': 1018,
  'humidity': 66,
  'sea_level': 1018,
  'grnd_level': 1017},
 'visibility': 10000,
 'wind': {'speed': 3.09, 'deg': 310},
 'clouds': {'all': 40},
 'dt': 1753454402,
 'sys': {'type': 2,
  'id': 2046553,
  'country': 'NL',
  'sunrise': 1753415405,
  'sunset': 1753472613},
 'timezone': 7200,
 'id': 2759794,
 'name': 'Amsterdam',
 'cod': 200}

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

1. Erreur 401 Unauthorized — API-clé:
    absent, incorrect, révoquée par website
2. Erreur 404 Not Found — Ville :
    Introuvable, mal orthographiée ou non disponible dans l’API.

**Comment gérer?**
    
     try:
        response = r.get(url, params=params)
        response.raise_for_status() # Vérifie si status code est 200, sinon lève une exception

        data = response.json()
        return data
    except r.HTTPError as http_err:
        print(f"Http Error: {http_err}")
    return None

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [ ]:
# Configuration Visual Crossing
load_dotenv() # charge les clés de .env dans les variables d'environnement
VC_API_KEY = os.getenv("VISUAL_CROSSING_API")
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date, api_key):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

    url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"

    params = {
        'key': api_key,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
    }

    # Votre implémentation ici
    # Pensez à la gestion d'erreurs !
    try:
        response = r.get(url, params=params)
        response.raise_for_status() # Vérifie si status code est 200, sinon lève une exception

        data = response.json()
        return data
    except r.HTTPError as http_err:
        print(f"Http Error: {http_err}")
    except r.exceptions.ConnectionError as cn_error:
        print(f"Error: {cn_error}")
    return None 

data = get_historical_weather("Montpellier", "2023-11-20", "2023-11-30", VC_API_KEY)

# if data and 'd' in data:
df_data = pd.DataFrame(data['days'])

df_data

,datetime,temp,humidity,precip,windspeed,pressure
0,2023-11-20,59.3,66.2,0.000,18.5,1017.7
1,2023-11-21,57.3,49.8,0.000,20.3,1013.5
2,2023-11-22,54.5,42.4,0.000,25.5,1017.6
3,2023-11-23,49.6,42.4,0.000,14.8,1022.6
4,2023-11-24,51.2,56.1,0.000,20.9,1014.3
5,2023-11-25,48.1,40.1,0.000,20.0,1016.0
6,2023-11-26,43.8,43.8,0.000,13.3,1017.5
7,2023-11-27,42.2,66.0,0.001,7.0,1011.7
8,2023-11-28,50.6,57.0,0.000,18.7,1006.4
9,2023-11-29,42.9,62.7,0.000,9.8,1008.5


### 🎯 Mission pratique
Récupérez les données des 30 derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [762]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
def get_air_quality(lat, lon, api_key):
    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """
    pass

In [763]:
### 🗺️ Obtenir les coordonnées
def get_city_coordinates(city, api_key):
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """
    pass


## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [764]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### �� API AgroMonitoring (gratuite)

In [765]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [766]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites